## I. Develop a Baseline Neural Network Mode



Đầu tiên tạo một baseline cho mô hình mạng nơ-ron cho bài toán hồi quy. Ta import những hàm và những đối tượng chúng ta cần:

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#load dataset
url = 'https://raw.githubusercontent.com/duyduc27/ML4_CBD_Robotics_course/master/study/Data%20files/housing.csv'
dataframe = pd.read_csv(url, delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input and output variables
X = dataset[:, :13] #input
Y = dataset[:, 13] #output
dataframe.head()

Using TensorFlow backend.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


- Load dữ liệu bộ dữ liệu từ file trên github xong thì tách ra input (X) và output (Y)<br>
- Bên dưới là định ra một hàm để tạo baseline cho mô hình được đánh giá. Đây là một mô hình đơn giản với một hidden layer fully connected, layer này có 13 nơ-ron bằng với số input attributes (13).  Mạng này dùng rectifier activation function ('relu') cho hidden layer. **Không có** activation function nào output layer vì đây là **bài toán hồi quy** và chúng ta quan tâm đến việc đưa ra tiên đoán giá trị dạng số mà không qua biến đổi.<br>
- Optimizer 'ADAM' được sử dụng và 'mean squared error' cho loss function. Đây cũng là metric để chúng ta sử dụng để đánh giá sự biểu diễn của mô hình.<br>
- Metric này được dùng bởi trung bình của bình phương sự khác biệt sai số cho ta thấy kết quả mà ta có thể trực tiếp hiểu được ngữ cảnh bài toán với đơn vị ở đây là hàng ngàn đô-la. 

In [0]:
# define base model
def baseline_model():
  # create model
  model = Sequential()
  model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

Seed để ta có cùng chung kết quả (chú ý khác phiên bản Keras có thể ra kết quả khác nhau). Estimator của Keras cho bài toán hồi quy là **KerasRegressor**.

In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [4]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print('Baseline %.2f (%.2f) MSE' % (results.mean(), results.std()))

W0705 08:40:41.949197 140254415796096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0705 08:40:41.991156 140254415796096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0705 08:40:42.001525 140254415796096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0705 08:40:42.035153 140254415796096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0705 08:40:42.200654 140254415796096 deprecation_wrapper.

Baseline -32.69 (23.47) MSE


MSE luôn là số dương nhưng đi qua mô hình bị đảo dấu. MSE tiến gần về 0 thì mô hình càng tốt. Ở ví dụ trên thì MSE là 32.69 và dao động ở 23.47

## II. Lift Performance By Standardizing The Dataset

Một vấn đề quan trọng cần quan tâm là bộ dữ liệu này có những input attributes khác nhau có tỷ lệ khác nhau vì chúng đo lường cho những đặc tính khác nhau. Một cách tốt để luyện tập là chuẩn bị lại dữ liệu trước khi lên mô hình. Tiếp tục từ baseline_model ở trên, ta có thể đánh giá cùng một mô hình với phiên bản input đã được chuẩn hóa dữ liệu.

In [0]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('Standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Standardized %.2f (%.2f) MSE' % (results.mean(), results.std()))

Standardized -22.60 (26.26) MSE


- Vậy ở ví dụ trên, sau khi chuẩn hóa thì MSE giảm xuống còn 22.6. Vậy là mô hình đã trở nên tốt hơn.
- Một cái mở rộng ở phần này là tương tự áp phần chuẩn hóa dữ liệu vào biến output để chuẩn hóa nó về range 0 - 1. Từ đó sử dụng activation function Sigmoid hay activation function tương tự để layer output để tiên đoán output có cùng range. (**không dùng** cho bài toán regression)

In [10]:
# Thử nghiệm với ý tưởng
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler

def baseline_model_2():
  # create model
  model = Sequential()
  model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

# evaluate model
estimators = []
estimators.append(('MinMaxScaler', MinMaxScaler(feature_range=(5,50))))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model_2, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

results = cross_val_score(pipeline, X, Y, cv=kfold)
print('MinMaxScaled: %.2f (%.2f)' % (results.mean(), results.std()))

MinMaxScaled: -28.90 (41.13)


Ở bài toán **Regression** không nên scale output Y bởi sẽ làm mất ý nghĩa đơn vị của output này. Sau một số lần thử nghiệm. Dùng MinMaxScaler biến input X về cùng tỷ lệ với output Y range(5, 50). Range được tìm ở dưới.<br>
Ta thấy MSE là 28.9 nhưng độ lệch chuẩn rất cao đến 41.13. Vậy dùng StandardScaler tốt hơn trong trường hợp này.

In [15]:
print('max:', np.amax(Y))
print('min:', np.amin(Y))

max: 50.0
min: 5.0


## III. Tune The Neural Network Topology

### 3.1 Evaluate a Deeper Network Topology

Ý tưởng tạo một mạng sâu hơn. Là thêm 1 lớp hidden layer. Ví dụ bên dưới là thêm 1 hidden layer với 6 neuron.<br>
13 inputs -> [13 -> 6] -> 1 output

In [16]:
# Regression Example With Boston Dataset: Standardized and Larger

# define model
def larger_model():
  #create model
  model = Sequential()
  model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('Standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Larger: %.2f (%.2f) MSE' % (results.mean(), results.std()))

Larger: -21.01 (24.85) MSE


Ta thấy kết quả của mô hình có tốt hơn khi vừa chuẩn hóa dữ liệu, vừa thêm một hidden layer. MSE đã được cải thiện!

## 3.2 Evaluate a Wider Network Topology

Ý tưởng là tăng kích thước của mô hình rộng hơn bằng việc tăng số nơ-ron. Ở ví dụ này ta giữ nguyên số layer nhưng tăng số neuron của hidden layer lên.<br>
13 inputs -> [20] -> 1 output

In [17]:
# Regression Example With Boston Dataset: Standardized and Wider

#define model
def wider_model():
  # create model
  model = Sequential()
  model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('Standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)

results = cross_val_score(pipeline, X, Y, cv=kfold)
print('Wider: %.2f (%.2f) MSE' % (results.mean(), results.std()))

Wider: -21.49 (25.12) MSE


Kết quả ở đây cũng tốt nhưng có vẻ không tốt bằng Deeper Network Topology.

## IV. Conclusions
- Một lần nữa việc chuẩn hóa dữ liệu đem lại kết quả tốt hơn cho mô hình Neuron Network (cho cả bài toán Classification và Regression).
- Sẽ khó để biết được liệu Deeper hay Wider Network Topology cái nào tốt hơn, ngoại trừ việc kiểm thử.
- Tuyệt đối không transform biến output Y ở bài toán Regression vì làm mất ý nghĩa đơn vị của bài toán này. Chưa kể là không thể ứng dụng được.